In [6]:
!pip install langchain-google-genai python-dotenv

In [16]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.schema import SystemMessage, HumanMessage, AIMessage
import os

from dotenv import load_dotenv
import os

load_dotenv(".env")  # Load environment variables
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
os.environ["GOOGLE_API_KEY"] = GEMINI_API_KEY


# Initialize the conversational agent with a free-tier model
chat_model = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0.2)  # Use 'gemini-1.5-flash' for faster responses

def drug_info_agent():
    print("\nWelcome to the Virtual interview Agent!  Type 'exit' to end.\n")
    conversation = [
        SystemMessage(content="You are an AI assistant specialized in taking interviews of data science candidates"
                      " You have to ask 5 questions and wait for the other person to reply and judge him on the basis of his answers and give hima final score out of 5"

                      "Please check for informal language clarification request unknown answers")
    ]

    while True:
        user_input = input("You: ")
        if user_input.lower() == "exit":
            print("Agent: Goodbye! Stay safe.")
            break

        conversation.append(HumanMessage(content=user_input))

        # Get agent's response using the correct method
        response = chat_model.invoke(conversation)
        conversation.append(AIMessage(content=response.content))

        print(f"Agent: {response.content}\n")

if __name__ == "__main__":
    drug_info_agent()



Welcome to the Virtual interview Agent!  Type 'exit' to end.

You: hi
Agent: Hi there!  Let's begin the data science interview. I'll ask you five questions, and please answer them as thoroughly and clearly as possible.  Ready?

You: yes
Agent: Okay, here's the first question:

**Question 1:**  Describe a situation where you had to deal with imbalanced classes in a dataset. What techniques did you use to address this issue, and why did you choose those specific techniques?  Explain your reasoning and the results you achieved.

You: fuck it
Agent: I understand that you may be feeling frustrated, but using that kind of language isn't appropriate for a professional interview setting.  We need to maintain a respectful and professional tone.  

Let's try again.  Please answer Question 1: Describe a situation where you had to deal with imbalanced classes in a dataset. What techniques did you use to address this issue, and why did you choose those specific techniques? Explain your reasoning a

KeyboardInterrupt: Interrupted by user

In [13]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.schema import SystemMessage, HumanMessage, AIMessage
import os
import fitz  # PyMuPDF for PDF reading
from dotenv import load_dotenv
from IPython.display import display, FileLink
from google.colab import files  # If using in Colab

# Load API Key from .env
load_dotenv(".env")
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
os.environ["GOOGLE_API_KEY"] = GEMINI_API_KEY

# Initialize AI model (free-tier model)
chat_model = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0.2)

def extract_text_from_pdf(pdf_path):
    """Extracts text from a given PDF file."""
    text = ""
    doc = fitz.open(pdf_path)
    for page in doc:
        text += page.get_text("text") + "\n"
    return text.strip()

def conduct_interview(job_role=None, pdf_path=None):
    """Conducts a virtual interview based on a specified job role or uploaded JD."""
    if pdf_path:
        job_description = extract_text_from_pdf(pdf_path)
        prompt = f"Generate 5 interview questions based on the following job description:\n{job_description}"
    elif job_role:
        prompt = f"Generate 5 interview questions for a {job_role} position."
    else:
        print("Please specify a job role or upload a JD.")
        return

    conversation = [
        SystemMessage(content="You are an AI interviewer. Ask 5 questions for the given role.")
    ]

    # Generate questions
    questions_response = chat_model.invoke([HumanMessage(content=prompt)])
    questions = questions_response.content.split("\n")
    print("\nAI Interviewer: Let's begin the interview!\n")

    total_score = 0

    for i, question in enumerate(questions, 1):
        print(f"Question {i}: {question}")
        user_answer = input("Your Answer: ")

        # AI Evaluation
        eval_prompt = f"Evaluate the following response for question: '{question}'. Response: '{user_answer}'. Give a score out of 5 and provide feedback."
        evaluation = chat_model.invoke([HumanMessage(content=eval_prompt)])

        print(f"AI Evaluation: {evaluation.content}\n")

        # Extract Score
        try:
            score = int([int(s) for s in evaluation.content.split() if s.isdigit()][0])
        except:
            score = 0  # Default to 0 if extraction fails

        total_score += score

    print(f"Final Score: {total_score} / 25")

# Upload JD if needed (for Google Colab)
def upload_pdf():
    uploaded = files.upload()
    pdf_path = list(uploaded.keys())[0]
    return pdf_path

# Example Usage
# For job role-based interview
conduct_interview(job_role="Data Scientist")

# For PDF-based interview (Colab users upload first)
# pdf_path = upload_pdf()
# conduct_interview(pdf_path=pdf_path)



AI Interviewer: Let's begin the interview!

Question 1: 1. **Describe a time you had to deal with messy or incomplete data. What steps did you take to clean and prepare it for analysis, and what were the challenges you encountered?**  (This assesses practical experience, problem-solving skills, and attention to detail.)
Your Answer: idk
AI Evaluation: Score: 0/5

Feedback:

This response is completely inadequate.  "idk" demonstrates a lack of preparation and an inability to answer a fundamental question about data analysis experience.  This is a crucial skill for many data-related roles, and the inability to articulate even one instance of dealing with messy data is a major red flag.  The candidate needs to provide a concrete example, outlining the specific data issues, the methods used to clean the data (e.g., handling missing values, outlier detection, data transformation), and the challenges faced (e.g., inconsistencies in data formats, data integrity issues, time constraints).  Th

In [14]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.schema import SystemMessage, HumanMessage, AIMessage
import os
import ipywidgets as widgets
from IPython.display import display
from dotenv import load_dotenv

# Load API Key
load_dotenv(".env")
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
os.environ["GOOGLE_API_KEY"] = GEMINI_API_KEY

# Initialize AI model
chat_model = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0.2)

class InterviewBot:
    def __init__(self, job_role="Data Science"):
        self.job_role = job_role
        self.questions = []
        self.answers = []
        self.scores = []
        self.current_question = 0
        self.feedback_widgets = []

    def start_interview(self):
        """Generate interview questions and begin the process."""
        prompt = f"Generate 5 interview questions for a {self.job_role} position."
        questions_response = chat_model.invoke([HumanMessage(content=prompt)])
        self.questions = [q for q in questions_response.content.split("\n") if q]

        print("\n🔹 Welcome to the AI Interview! Answer the questions below:\n")
        self.ask_question()

    def ask_question(self):
        """Ask the next question using a widget."""
        if self.current_question < len(self.questions):
            q_text = self.questions[self.current_question]
            print(f"\n**Question {self.current_question + 1}:** {q_text}")

            ans_box = widgets.Text(placeholder="Type your answer here...")
            submit_button = widgets.Button(description="Submit Answer")
            feedback_label = widgets.Label(value="")

            def process_answer(b):
                user_answer = ans_box.value
                eval_prompt = f"Evaluate the response for question: '{q_text}'. Response: '{user_answer}'. Give a score out of 5 and provide feedback."
                evaluation = chat_model.invoke([HumanMessage(content=eval_prompt)])

                # Extract score
                try:
                    score = int([int(s) for s in evaluation.content.split() if s.isdigit()][0])
                except:
                    score = 0

                self.answers.append(user_answer)
                self.scores.append(score)
                feedback_label.value = f"⭐ AI Feedback: {evaluation.content}\n"

                # Move to next question
                self.current_question += 1
                if self.current_question < len(self.questions):
                    self.ask_question()
                else:
                    self.show_final_score()

            submit_button.on_click(process_answer)
            display(ans_box, submit_button, feedback_label)
            self.feedback_widgets.append(feedback_label)
        else:
            self.show_final_score()

    def show_final_score(self):
        """Calculate and display the final score."""
        final_score = sum(self.scores)
        print(f"\n🎯 **Final Score: {final_score} / 25**")
        print("\n📌 **Feedback & Tips for Improvement:**")

        eval_summary_prompt = f"Candidate scored {final_score} out of 25. Provide constructive feedback on areas for improvement."
        summary_feedback = chat_model.invoke([HumanMessage(content=eval_summary_prompt)])
        print(summary_feedback.content)

# Start Interview
interview = InterviewBot(job_role="Data Science")
interview.start_interview()



🔹 Welcome to the AI Interview! Answer the questions below:


**Question 1:** 1. **Describe a time you had to deal with messy or incomplete data.  What steps did you take to clean and prepare it for analysis, and what were the challenges you encountered?** (This assesses practical experience, problem-solving skills, and attention to detail.)


Text(value='', placeholder='Type your answer here...')

Button(description='Submit Answer', style=ButtonStyle())

Label(value='')


**Question 2:** 2. **Explain a statistical concept you find particularly useful in your work, and provide a concrete example of how you've applied it to solve a real-world problem.** (This tests their understanding of core statistical principles and their ability to connect theory to practice.)


...


**Question 5:** 5. **Let's say your model is performing poorly after deployment.  Describe your troubleshooting process.  What steps would you take to identify the root cause and improve its performance?** (This explores their ability to handle real-world challenges and their problem-solving skills in a production environment.)


Text(value='', placeholder='Type your answer here...')

Button(description='Submit Answer', style=ButtonStyle())

Label(value='')


🎯 **Final Score: 0 / 25**

📌 **Feedback & Tips for Improvement:**
A score of 0 out of 25 indicates a significant gap in understanding and performance.  The feedback needs to be tailored to the specific assessment, but here are some general areas for improvement, categorized for clarity:

**I. Understanding the Fundamentals:**

* **Content Knowledge:**  The candidate lacks a foundational understanding of the core concepts tested.  Review the learning materials thoroughly, focusing on key definitions, principles, and theories.  Identify specific areas of weakness and seek clarification from instructors or mentors.  Consider using different learning resources like textbooks, online tutorials, or practice questions.
* **Instructions & Question Interpretation:**  The candidate may have struggled to understand the questions or the instructions provided.  Practice carefully reading and interpreting questions, identifying keywords and what is being asked.  Work on breaking down complex questi